In [80]:
import plotly.graph_objects as go
import os
import pandas as pd
from config import *

In [81]:
timestamp = "2021-06-30_15-00-00"

latest_run = True

In [82]:
# List all folders in the directory
runs = [f for f in os.listdir("csv") if os.path.isdir(os.path.join("csv", f))]

if latest_run:
    timestamp = sorted(runs)[-1]

df_game_data = pd.read_csv(f"csv/{timestamp}/game_data.csv", delimiter=";")
df_bid_logs = pd.read_csv(f"csv/{timestamp}/bid_logs.csv", delimiter=";")

df_ask_prices = pd.read_csv(f"csv/{timestamp}/ask_prices.csv", delimiter=";")
df_bid_prices = pd.read_csv(f"csv/{timestamp}/bid_prices.csv", delimiter=";")
df_ask_agg = pd.read_csv(f"csv/{timestamp}/ask_agg.csv", delimiter=";")
df_bid_agg = pd.read_csv(f"csv/{timestamp}/bid_agg.csv", delimiter=";")
df_target_price_param = pd.read_csv(f"csv/{timestamp}/target_price_param.csv", delimiter=";")
df_limit_buy = pd.read_csv(f"csv/{timestamp}/limit_buy.csv", delimiter=";")
df_limit_sell = pd.read_csv(f"csv/{timestamp}/limit_sell.csv", delimiter=";")
df_target_asks = pd.read_csv(f"csv/{timestamp}/target_asks.csv", delimiter=";")
df_target_bids = pd.read_csv(f"csv/{timestamp}/target_bids.csv", delimiter=";")

df_market_positions = pd.read_csv(f"csv/{timestamp}/market_positions.csv", delimiter=";")
df_payoffs = pd.read_csv(f"csv/{timestamp}/payoffs.csv", delimiter=";")
df_revenues = pd.read_csv(f"csv/{timestamp}/revenues.csv", delimiter=";")
df_expenses = pd.read_csv(f"csv/{timestamp}/expenses.csv", delimiter=";")
df_prod_costs = pd.read_csv(f"csv/{timestamp}/prod_costs.csv", delimiter=";")
df_penalty_imbalances = pd.read_csv(f"csv/{timestamp}/penalty_imbalances.csv", delimiter=";")
df_imbalances = pd.read_csv(f"csv/{timestamp}/imbalances.csv", delimiter=";")

colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown']

In [83]:
df_bid_logs

,Unnamed: 0,bidder,price,volume,transaction_price,match_flag
0,1,0.0,19.710679,2000.000000,NaN,ask
1,2,2.0,18.182955,2000.000000,NaN,ask
2,3,2.0,17.055479,2000.000000,NaN,ask
3,4,0.0,17.035665,2000.000000,NaN,ask
4,5,0.0,17.019926,2000.000000,NaN,ask
...,...,...,...,...,...,...
194,195,3.0,13.879697,34.933206,NaN,ask
195,196,2.0,1.773421,1.452824,NaN,bid
196,197,3.0,13.879697,34.668615,NaN,ask
197,198,1.0,1.786679,0.447868,NaN,bid


In [84]:
df_imbalances

,Unnamed: 0,bidder_0,bidder_1,bidder_2,bidder_3,bidder_4,bidder_5
0,1,-513.191007,-362.150518,-25.774545,-8.855172,-61.181211,-443.225453
1,2,-513.191007,-362.150518,-25.774545,-8.855172,-61.181211,-443.225453
2,3,80.409675,-352.578745,-3.147253,-6.866891,-13.016511,-420.579484
3,4,226.944043,-332.086903,-40.364318,0.000000,-100.879488,-429.733942
4,5,83.153586,-321.816629,-37.225092,0.000000,-68.468626,-522.092870
...,...,...,...,...,...,...,...
194,195,-195.508194,-1.350870,-0.494600,0.000000,-3.729212,-1.319677
195,196,-199.819949,-0.662125,-1.452824,0.000000,-1.796679,-0.694605
196,197,-201.912330,-0.475658,-1.612288,0.000000,-0.925454,-1.950197
197,198,-210.597754,-0.447868,-1.618145,0.000000,-1.681603,-1.537641


In [85]:
df_ask_agg

,Unnamed: 0,bidder_0,bidder_1,bidder_2,bidder_3,bidder_4,bidder_5
0,1,-0.780121,-0.359332,-0.142936,-0.397489,-0.630842,-0.178668
1,2,-0.780121,-0.359332,-0.142936,-0.397489,-0.630842,-0.178668
2,3,-0.352273,-0.359332,-0.142936,-0.397489,-0.399726,-0.178668
3,4,-0.309731,-0.305333,-0.142936,-0.301786,-0.305752,-0.178668
4,5,-0.309731,-0.305333,-0.142936,-0.301786,-0.305752,-0.178668
...,...,...,...,...,...,...,...
194,195,-0.142947,-0.142946,-0.142936,-0.142947,-0.142947,-0.142947
195,196,-0.142947,-0.142946,-0.142936,-0.142947,-0.142947,-0.142947
196,197,-0.142947,-0.142946,-0.142936,-0.142947,-0.142947,-0.142947
197,198,-0.142947,-0.142946,-0.142936,-0.142947,-0.142947,-0.142947


In [86]:
df_bid_agg

,Unnamed: 0,bidder_0,bidder_1,bidder_2,bidder_3,bidder_4,bidder_5
0,1,-0.752271,-0.988665,-0.326247,-0.877992,-0.554266,-0.11069
1,2,-0.752271,-0.988665,-0.326247,-0.877992,-0.554266,-0.11069
2,3,-0.752271,-0.988665,-0.326247,-0.877992,-0.554266,-0.11069
3,4,-0.752271,-0.988665,-0.326247,-0.877992,-0.554266,-0.11069
4,5,-0.752271,-0.988665,-0.326247,-0.877992,-0.554266,-0.11069
...,...,...,...,...,...,...,...
194,195,-0.408017,-0.407004,-0.326247,-0.248049,-0.410449,-0.11069
195,196,-0.408017,-0.407004,-0.326247,-0.248049,-0.410449,-0.11069
196,197,-0.383794,-0.382717,-0.326247,-0.222526,-0.384961,-0.11069
197,198,-0.383794,-0.382717,-0.326247,-0.222526,-0.384961,-0.11069


In [87]:
df_game_data

,Unnamed: 0,market,marginal_price,social_welfare,system_imbalance,imbalance_penalty_factor,transaction_price,top_bid,top_ask,equilibrium_price_estimate,volatility,last_event,last_price
0,0,DA,9.0,31155.590234,0.000000,1.000000,9.0,NaN,NaN,9.0,0.0,match,9.000000
1,1,ID,NaN,NaN,-1119.257168,1.023293,NaN,0.000000,19.710679,9.0,0.0,ask,19.710679
2,2,ID,NaN,NaN,-1414.377907,1.047129,NaN,0.000000,18.182955,9.0,0.0,ask,18.182955
3,3,ID,NaN,NaN,-715.779208,1.071519,NaN,0.000000,17.055479,9.0,0.0,ask,17.055479
4,4,ID,NaN,NaN,-676.120608,1.096478,NaN,0.000000,17.035665,9.0,0.0,ask,17.035665
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,ID,NaN,NaN,-202.402553,89.125094,NaN,1.580028,13.879697,9.0,0.0,ask,13.879697
196,196,ID,NaN,NaN,-204.426182,91.201084,NaN,1.773421,13.879697,9.0,0.0,bid,1.773421
197,197,ID,NaN,NaN,-206.875927,93.325430,NaN,1.773421,13.879697,9.0,0.0,ask,13.879697
198,198,ID,NaN,NaN,-215.883011,95.499259,NaN,1.786679,13.879697,9.0,0.0,bid,1.786679


In [88]:
### Bid-Ask spread

fig = go.Figure()

# Add lines for 'bid'
fig.add_trace(go.Scatter(
    #x=bid_data['index'],
    y=df_game_data['top_bid'],
    mode='lines',
    name='Top bid',
    line=dict(width=2)  # Adjust line width if necessary
))

# Add lines for 'ask'
fig.add_trace(go.Scatter(
    y=df_game_data['top_ask'],
    mode='lines',
    name='Top Ask',
    marker=dict(size=8)  # Adjust point size if necessary
))

# Add lines for 'Equilibrium price'
fig.add_trace(go.Scatter(
    #x=bid_data['index'],
    y=df_game_data['equilibrium_price_estimate'],
    mode='lines',
    name='Equilibrium price',
    line=dict(width=2)  # Adjust line width if necessary
))

# Add points for 'matches'
fig.add_trace(go.Scatter(
    y=df_game_data['transaction_price'],
    mode='markers',
    name='Matches',
    marker=dict(size=8)  # Adjust point size if necessary
))

# Update layout
fig.update_layout(
    title='Bid-Ask spread',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Type'
)

# Show the plot
fig.show()


In [89]:
# Payoffs
fig_payoffs = go.Figure()
for i in range(0, n-1):
    fig_payoffs.add_trace(go.Scatter(x=df_payoffs.index, y=df_payoffs[f'bidder_{i}'], mode='lines', name=f'bidder_{i}'))
fig_payoffs.update_layout(title='Payoffs')
fig_payoffs.show()



In [90]:
# Imbalances
fig_imb = go.Figure()
for i in range(0, n-1):
    fig_imb.add_trace(go.Scatter(x=df_imbalances.index, y=df_imbalances[f'bidder_{i}'], mode='lines', name=f'bidder_{i}'))
fig_imb.update_layout(title='Payoffs')
fig_imb.show()

In [91]:
# Prices
fig_prices = go.Figure()
for i in range(0, n-1):
    fig_prices.add_trace(go.Scatter(x=df_ask_prices.index, y=df_ask_prices[f'bidder_{i}'], line=dict(dash='dash'), mode='lines', name=f'ask_bidder_{i}'))
    fig_prices.add_trace(go.Scatter(x=df_bid_prices.index, y=df_bid_prices[f'bidder_{i}'], mode='lines', name=f'bid_bidder_{i}'))
fig_prices.update_layout(title='Prices')
fig_prices.show()